In [4]:
# Load relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn import preprocessing
sns.set()
sns.set_style('darkgrid')
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

path ='P:/GitHub/General-Assembly-Projects/Capstone/datasets/'

## Perform initial EDA on ratings dataset

In [2]:
# Load source data
df = pd.read_csv(path + 'ratings.csv')

In [3]:
def eda(df):
    """Given dataframe, generate exploratory data analysis"""
    # check that input is pandas dataframe
    if type(df) != pd.core.frame.DataFrame:
        raise TypeError("Only pandas dataframe is allowed as input")
        
    # replace field that's entirely space (or empty) with NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)

    print("Preview of data:")
    display(df.head(3))

    print("\nTo check: \n (1) Total number of entries \n (2) Column types \n (3) Any null values\n")
    print(df.info())

    # generate preview of entries with null values
    if len(df[df.isnull().any(axis=1)] != 0):
        print("\nPreview of data with null values:")
        display(df[df.isnull().any(axis=1)].head(3))
        missingno.matrix(df)
        plt.show()
        # Generate a heatmap to show missing data
        sns.heatmap(df.isnull(), cbar=False)

    # generate count statistics of duplicate entries
    if len(df[df.duplicated()]) > 0:
        print("\n***Number of duplicated entries: ", len(df[df.duplicated()]))
        display(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head())
        
        # To delete dupolicates, run the following;
        # df.drop_duplicates(inplace=True)
    else:
        print("\nNo duplicated entries found")

In [4]:
eda(df)

Preview of data:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224



To check: 
 (1) Total number of entries 
 (2) Column types 
 (3) Any null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None

No duplicated entries found


In [6]:
# make the user ids go from 0...N-1
df.userId = df.userId - 1

In [17]:
df.userId.value_counts()

413    2698
598    2478
473    2108
447    1864
273    1346
       ... 
405      20
193      20
430      20
52       20
575      20
Name: userId, Length: 610, dtype: int64

In [12]:
# Check how many movie 
df.movieId.value_counts()

356       329
318       317
296       307
593       279
2571      278
         ... 
5986        1
100304      1
34800       1
83976       1
8196        1
Name: movieId, Length: 9724, dtype: int64

In [11]:
# movie ids
df.movieId.values

array([     1,      3,      6, ..., 168250, 168252, 170875], dtype=int64)

In [7]:
# Creat a mapping for movie_ids
unique_movie_ids = set(df.movieId.values)
movie2idx = {}
count = 0
for movie_id in unique_movie_ids:
  movie2idx[movie_id] = count
  count += 1

In [8]:
# Assign movie index to a new column
df['movie_idx'] = df.apply(lambda row: movie2idx[row.movieId], axis=1)

df = df.drop(columns=['timestamp'])

In [15]:
df

,userId,movieId,rating,movie_idx
0,0,1,4.0,0
1,0,3,4.0,2
2,0,6,4.0,5
3,0,47,5.0,46
4,0,50,5.0,49
...,...,...,...,...
100831,609,166534,4.0,2291
100832,609,168248,5.0,3714
100833,609,168250,5.0,3716
100834,609,168252,5.0,3718


In [18]:
# Save edited ratings
df.to_csv(path + 'edited_ratings.csv', index = False)

## Preprocessing of selected users and items

In [19]:
import pickle
import numpy as np
import pandas as pd
from collections import Counter

In [20]:
# df = pd.read_csv('.\data\\MovieLens\\edited_rating.csv')
df = pd.read_csv(path + 'edited_ratings.csv')

In [21]:
print('Original dataframe size:', len(df))

Original dataframe size: 100836


In [23]:
# N = number of users; M = number of movies
N = df.userId.max() + 1 # Get the number of users + 1 because the range starts from 0
M = df.movie_idx.max() + 1 # Get the number of movies
print(f'num_of_users: {N}\nnum_of_movies: {M}')

# Number of user/ movie id count and store the count by user/ movie id in a dictionary format.
user_ids_count = Counter(df.userId)
movie_ids_count = Counter(df.movie_idx)

num_of_users: 610
num_of_movies: 9724


If we have a very powerful and fast machine at our disposal theoretically, we do not need to reduce the number of movies and users for our model. That is, we will search through all movies and users' ratings on every search. But in view of processing power limitation, we set a max number of movies and users ratings that can yield a reasonable accuracy.

In [24]:
# Set the number of users (n) and the number of movies (m) we would like to keep.
n = 10000 # Set the max nummber of users for the recommender model to process
m = 2000  # Set the max number of movies for the recommender model to process

In [25]:
# Replace num_of_users with n for the recommender model to work with to speed up performance 
user_ids = [u for u, c in user_ids_count.most_common(n)]

# Replace num_of_movies with m for the recommender model to work with to speed up performance 
movie_ids = [m for m, c in movie_ids_count.most_common(m)]

print(len(user_ids))
print(len(movie_ids))

610
2000


In [26]:
# In the event, a smaller dataset of user/ movie ids is required to boost performance,
# ie, the number of user/ movie ids are restricted to a certain number of size (n and m),
# get the user/ movie ids and put the details in a dataframe
# This is equivalent to SQL statement -
# select * from dataframe where user_id in users_ids and movie_id om movie_ids

# Create a new dataframe to store the results
df_small = df[df.userId.isin(user_ids) & df.movie_idx.isin(movie_ids)].copy()

In [27]:
df_small

,userId,movieId,rating,movie_idx
0,0,1,4.0,0
1,0,3,4.0,2
2,0,6,4.0,5
3,0,47,5.0,46
4,0,50,5.0,49
...,...,...,...,...
100818,609,159093,3.0,9063
100829,609,164179,5.0,326
100830,609,166528,4.0,2285
100833,609,168250,5.0,3716


In [28]:
# Need to remake user ids and movie ids since they are no longer sequential
new_user_id_map = {}
i = 0
for old in user_ids:
    new_user_id_map[old] = i
    i += 1
print("i:", i)

i: 610


In [29]:
new_movie_id_map = {}
j = 0
for old in movie_ids:
    new_movie_id_map[old] = j
    j += 1
print("j:", j)  

j: 2000


In [30]:
print("Setting new ids")
df_small.loc[:, 'userId'] = df_small.apply(lambda row: new_user_id_map[row.userId], axis=1)
df_small.loc[:, 'movie_idx'] = df_small.apply(lambda row: new_movie_id_map[row.movie_idx], axis=1)

Setting new ids


In [31]:
print("max user id:", df_small.userId.max())
print("max movie id:", df_small.movie_idx.max())

max user id: 609
max movie id: 1999


In [33]:
print("small dataframe size:", len(df_small))
df_small.to_csv(path + 'smaller_ratings_dataset.csv', index=False)

small dataframe size: 78305


In [34]:
df_small

,userId,movieId,rating,movie_idx
0,111,1,4.0,11
1,111,3,4.0,418
2,111,6,4.0,127
3,111,47,5.0,15
4,111,50,5.0,13
...,...,...,...,...
100818,5,159093,3.0,1996
100829,5,164179,5.0,988
100830,5,166528,4.0,945
100833,5,168250,5.0,1631


# Preprocess to Dictionary

In code, questions like;
- Given user i, which movies j did they rate?
- Given movie j, which users have rated it?
- Given user i and movie j, what is the rating?

Theoretically, Pandas dataframe is like an SQL table, so we should be able to write 'queries' to grab this info but we are not sure of how fast can Pandas return results.<br>

However,wWe can use Python dictionary to handle key/ value look up;
- for user to movies lookup (user id -> movie id): the key is the user id and the value is a list of movie ids for the movies they rated.
- for movie to users lookup(movie id -> user id): the key is the movie id and the value is a list of user ids for the users who have rated the movie
- for ratings lookup (user id, movie id) -> rating: the key is the tuple of user id and movie id and the value is the rating

#### Why dictionaries?
When we loop through to find the ratings, we are looping through the entire ratings matrix and look for the values that are populated, that is going to be O(NM), but if we have a dictionary that only contains existing ratings then every loop through those dictionary, it's always O of the size of Omega where Omega is the set of ratings.


In [35]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [36]:
# load in the data
df = pd.read_csv(path + 'smaller_ratings_dataset.csv')

In [40]:
N = df.userId.max() + 1 # Number of users
M = df.movie_idx.max() + 1 # Number of movies

# Split into train/ test sets
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

In [41]:
# A dictionary to tell us which users have rated whcih movies
user2movie = {}

# A dictionary to tell us which movies have been rated by which users
movie2user = {}

# A dictionary to look up ratings
usermovie2rating = {}
print('Calling: update_user2movie_and_movie2user')
count = 0

# Function that accept a single row of a dataframe and base on the row is going to add
# an entyr to each of these dictionaries. Since the function operates on a row, we can use
# dataframe.apply() to call this function.

def update_user2movie_and_movie2user(row):
    global count # This is for debugging purpose as this function can take some time to run
    count += 1
    if count % 100000 == 0:
        print(f'processed: {float(count)/ cutoff}')
    
    i = int(row.userId)
    j = int(row.movie_idx)
    if i not in user2movie:
        user2movie[i] = [j] # Create a new list with only j as the single movie user i has rated
    else:
        user2movie[i].append(j) # Append movie_idx to user i's existing movie list
        
    if j not in movie2user:
        movie2user[j] = [i] # Create a new list with only i as the user that has rated the movie
    else:
        movie2user[j].append(i) # Append user id to movie j's existing user list
    
    # Tuple as the key in to the ratings dictionary with the ratings as the value
    usermovie2rating[(i,j)] = row.rating
    
df_train.apply(update_user2movie_and_movie2user, axis=1)

Calling: update_user2movie_and_movie2user


43455    None
66965    None
8029     None
39222    None
66177    None
         ... 
20171    None
65214    None
73438    None
54591    None
52211    None
Length: 62644, dtype: object

In [42]:
# Test ratings dictionary
usermovie2rating_test = {}
print("Calling: update_usermovie2rating_test")
count = 0

def update_usermovie2rating_test(row):
    global count
    count += 1
    if count % 100000 == 0:
        print(f'processed: {float(count)/ len(df_test)}')
    
    i = int(row.userId)
    j = int(row.movie_idx)
    usermovie2rating_test[(i,j)] = row.rating

df_test.apply(update_usermovie2rating_test, axis=1)

Calling: update_usermovie2rating_test


69569    None
11915    None
23633    None
4177     None
30901    None
         ... 
36939    None
75278    None
2463     None
49984    None
38768    None
Length: 15661, dtype: object

In [43]:
# Note that, data are saved in binary format (ie, python dictionaries)
# instead of json although the file extension is .json. This is to retain the format
# for easier loading later on.

with open(path + 'user2movie.json', 'wb') as f:
    pickle.dump(user2movie,f)

with open(path + 'movie2user.json', 'wb') as f:
    pickle.dump(movie2user,f)

with open(path + 'usermovie2rating.json', 'wb') as f:
    pickle.dump(usermovie2rating,f)
    
with open(path + 'usermovie2rating_test.json', 'wb') as f:
    pickle.dump(usermovie2rating_test,f)

# User to User Collaborative

In [44]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

In [45]:
# Load in the data
if not os.path.exists(path + 'user2movie.json') or \
    not os.path.exists(path + 'movie2user.json') or \
    not os.path.exists(path + 'usermovie2rating.json') or \
    not os.path.exists(path + 'usermovie2rating_test.json'):
    import preprocess2dict
    
with open(path + 'user2movie.json', 'rb') as f:
    user2movie = pickle.load(f)

with open(path + 'movie2user.json', 'rb') as f:
    movie2user = pickle.load(f)

with open(path + 'usermovie2rating.json', 'rb') as f:
    usermovie2rating = pickle.load(f)
    
with open(path + 'usermovie2rating_test.json', 'rb') as f:
    usermovie2rating_test = pickle.load(f)


In [46]:
N = np.max(list(user2movie.keys())) + 1
# The test set may contain movies the train set doesn't have data on
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

#if N > 10_000:
#    print('N = ', N, ' are you sure you want to continue?')
#    print('Comment out these lines if so...')
#    exit()

N: 610 M: 2000


In [48]:
# To find the similarities, we have to do O(N^2 * M) calculations!
# If possible, parallelize the calculations to reduce run time
# Note: We really only have to do half the calculations, since w_ij is symmetric
K = 25 # Number of neighbors we'd like to consider
limit = 5 # Number of common movies users must have in commmon in order to consider
neighbors = [] # Store neighbors in this list
averages = [] # Each user's average rating for later use
deviations = [] # Each user's deviation for later use
for i in range(N):
    # Find the 25 closest users to user i
    movies_i = user2movie[i]
    movies_i_set = set(movies_i)
    
    # Calculate average and deviation
    ratings_i = { movie:usermovie2rating[(i, movie)] for movie in movies_i }
    avg_i = np.mean(list(ratings_i.values()))
    dev_i = { movie:(rating - avg_i) for movie, rating in ratings_i.items() }
    dev_i_values = np.array(list(dev_i.values()))
    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))
    
    # Save these for later use
    averages.append(avg_i)
    deviations.append(dev_i)
    
    sl = SortedList()
    for j in range(N):
        # Don't include ourselves
        if j != i:
            movies_j = user2movie[j]
            movies_j_set = set(movies_j)
            common_movies = (movies_i_set & movies_j_set) # intersection
            if len(common_movies) > limit:
                # Calculate average & deviation
                ratings_j = { movie:usermovie2rating[(j, movie)] for movie in movies_j }
                avg_j = np.mean(list(ratings_j.values()))
                dev_j = { movie:(rating - avg_j) for movie, rating in ratings_j.items() }
                dev_j_values = np.array(list(dev_j.values()))
                sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))
                
                # Calculate correlation coefficient
                numerator = sum(dev_i[m]*dev_j[m] for m in common_movies)
                w_ij = numerator / (sigma_i * sigma_j)
                
                # Insert into sorted list and truncate
                # Negate weight, because list is sorted ascending
                # Maximum value (1) is 'closest'
                sl.add((-w_ij, j))
                if len(sl) > K:
                    del sl[-1]

    # Store the neighbors
    neighbors.append(sl)
    
    # Print out useful things
    if i % 1 == 0:
        print(i)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [50]:
# Using neighbors, calculate train and test MSE
def predict(i, m):
    # Calculate the weighted sum of deviations
    numerator = 0
    denominator = 0
    for neg_w, j in neighbors[i]:
        # Remember the weight is stored as its negative
        # so the negative of the negative weight is the positive weight
        try:
            numerator += -neg_w * deviations[j][m]
            denominator += abs(neg_w)
        except KeyError:
            # Neighbors may not have rated the same movie
            # don't want to do dictionary lookup twive
            # so just throw away exception
            pass
            
    if denominator == 0:
        prediction = averages[i]
    else:
        prediction = numerator / denominator + averages[i]
    prediction = min(5, prediction)
    prediction = max(0.5, prediction) # min rating is 0.5
    return prediction

In [51]:
train_predictions = []
train_targets = []
for (i, m), target in usermovie2rating.items():
    # calculate the prediction for this movie
    prediction = predict(i, m)
        
    # Save the prediction and target
    train_predictions.append(prediction)
    train_targets.append(target)
        
test_predictions = []
test_targets = []
# Same thing for test set
for (i, m), target in usermovie2rating_test.items():
    # calculate the prediction for this movie
    prediction = predict(i, m)

    # save the prediction and target
    test_predictions.append(prediction)
    test_targets.append(target)


In [53]:
# Calculate accuracy
def mse(p, t):
    p = np.array(p)
    t = np.array(t)
    return np.mean((p - t) ** 2)
    
print('train mse: ', mse(train_predictions, train_targets))
print('test mse: ', mse(test_predictions, test_targets))

train mse:  0.6060294591616198
test mse:  0.7890812691580539


In [54]:
for (i, m), target in usermovie2rating_test.items():
        # Calculate the prediction for this movie
    print(i, m, target)

136 61 4.0
34 962 1.0
77 347 4.0
164 75 3.0
503 238 5.0
158 610 5.0
44 935 4.0
180 1 4.5
46 585 3.0
106 63 4.0
45 184 4.0
20 523 4.0
235 298 2.5
54 98 4.0
347 208 4.0
0 1310 3.5
53 1458 5.0
13 283 4.5
178 1009 4.0
31 1000 3.0
131 174 4.0
18 426 4.0
118 82 5.0
174 126 3.5
285 1 4.0
349 59 3.0
135 67 3.0
277 368 3.0
261 77 4.0
40 33 4.0
176 156 4.5
176 480 4.0
8 115 4.0
229 144 4.0
110 21 5.0
177 1793 4.5
35 1366 3.0
67 88 4.0
478 1945 4.0
22 430 3.5
84 148 4.5
62 198 5.0
584 1128 3.5
8 100 4.0
584 881 3.5
313 304 3.0
159 934 4.0
16 1706 3.0
37 344 4.5
170 1558 5.0
275 44 4.0
20 796 3.0
202 1076 3.0
27 818 3.0
1 1156 2.5
104 1443 2.5
164 17 3.0
185 199 2.0
0 1929 5.0
513 23 5.0
288 283 4.0
74 1289 3.0
154 435 4.0
1 471 3.0
201 553 4.0
71 155 1.0
279 234 4.5
359 19 3.5
114 50 4.0
228 1548 3.0
51 28 2.0
116 185 4.0
402 38 5.0
19 1986 4.0
370 483 4.0
263 1667 2.0
43 270 4.5
337 327 4.0
461 53 5.0
3 345 3.0
19 130 4.0
132 31 4.0
509 23 3.0
339 463 3.0
99 207 3.0
2 1432 4.0
454 84 3.0
13 754 

38 161 3.5
15 891 0.5
108 1272 4.5
33 326 2.0
535 18 4.0
67 111 4.0
230 18 4.5
563 17 5.0
127 324 3.0
7 400 5.0
105 44 3.5
21 695 2.0
25 347 3.0
52 468 3.0
79 1179 5.0
122 162 4.0
105 815 3.0
47 1105 3.5
232 116 5.0
99 136 3.5
7 1230 4.0
150 14 3.0
239 1085 5.0
80 84 4.0
40 725 4.5
4 1255 3.5
132 1949 4.0
15 89 3.0
97 451 4.0
331 1218 4.0
78 683 4.5
142 263 4.0
207 154 5.0
248 204 4.0
76 452 3.0
402 115 3.5
120 146 4.0
61 32 4.5
62 470 4.0
82 1999 3.5
168 143 3.5
39 696 4.0
463 131 3.0
54 1623 4.0
145 432 3.0
110 170 4.0
4 297 2.0
49 159 4.0
35 537 3.0
504 1733 3.0
0 1645 4.0
246 127 5.0
6 1027 2.5
249 0 3.5
24 67 3.5
162 236 4.0
168 46 3.5
218 0 5.0
6 10 2.5
2 1592 4.0
1 538 3.5
348 43 5.0
151 380 3.5
68 841 5.0
549 587 2.0
314 4 5.0
53 227 4.0
18 238 3.5
80 27 5.0
8 417 3.0
119 8 3.5
6 1486 2.0
32 1341 3.0
202 271 4.0
48 664 3.0
69 1 4.0
188 398 1.5
86 30 3.0
292 1654 4.0
71 608 1.0
207 435 3.0
102 5 4.0
39 903 4.0
83 1748 1.0
130 617 3.0
32 6 4.0
258 23 5.0
22 412 3.0
87 122 5.0
30 

34 120 3.0
41 70 4.5
89 335 5.0
161 412 2.0
593 1297 3.0
121 141 2.0
563 112 4.0
14 190 5.0
250 861 3.5
69 634 4.5
0 903 3.0
337 1086 2.5
26 601 4.0
287 47 3.0
128 325 3.0
181 1307 3.0
1 1683 1.5
21 1298 1.0
41 328 4.0
24 14 5.0
61 1008 3.0
110 499 0.5
115 83 1.5
138 2 5.0
30 695 3.0
91 174 3.0
133 1106 4.0
21 50 4.0
95 18 5.0
43 1094 0.5
120 694 3.0
16 37 3.5
299 283 1.0
241 54 3.0
6 564 4.0
3 172 2.0
32 1516 4.0
0 699 3.0
21 788 4.5
406 208 3.0
45 335 4.0
5 538 4.0
187 596 3.5
82 132 2.5
8 161 3.0
36 1528 4.0
7 604 5.0
382 521 4.0
22 336 2.5
221 565 3.5
9 796 4.0
271 236 3.0
343 1533 4.0
375 1058 2.5
16 879 4.5
45 31 3.5
281 79 5.0
8 1166 4.0
99 197 3.5
158 458 4.0
48 511 4.0
144 1010 3.0
164 125 3.0
56 1254 1.0
119 33 4.0
191 314 3.0
7 11 5.0
55 330 4.0
89 146 5.0
39 164 4.0
112 4 5.0
55 199 1.0
84 25 4.5
91 154 5.0
7 695 3.0
55 174 4.0
16 850 4.0
27 44 2.0
87 150 5.0
565 1576 5.0
597 424 5.0
99 998 4.0
220 13 4.5
388 248 5.0
86 96 3.0
23 805 1.5
24 1266 4.0
340 166 4.0
114 976 2.0


275 100 4.5
382 60 5.0
572 38 4.0
220 1825 3.5
21 657 3.5
354 24 4.0
39 374 5.0
52 1206 4.5
249 19 4.0
380 27 3.0
267 413 3.0
5 253 5.0
540 863 5.0
29 482 4.0
72 1697 2.5
171 297 1.0
36 331 3.5
36 804 2.5
26 1493 3.0
51 350 4.0
32 1019 4.5
99 1027 2.0
377 143 4.0
163 1870 4.0
30 289 3.0
261 458 3.5
19 107 3.5
198 426 4.0
1 1231 1.5
222 26 3.5
596 977 5.0
267 8 5.0
25 733 1.0
51 111 4.0
14 1358 4.0
152 499 3.0
476 497 5.0
44 347 2.0
3 1013 5.0
38 215 4.0
79 1505 3.0
5 14 3.5
3 1738 1.5
93 903 4.0
35 1037 3.0
353 49 3.0
7 178 5.0
46 887 3.0
42 95 3.0
205 133 4.0
1 550 2.5
51 2 3.0
65 1498 4.0
496 1745 3.0
457 52 3.0
113 427 4.0
476 19 5.0
14 2 5.0
80 560 4.0
54 93 2.0
75 335 4.5
2 160 4.0
19 64 4.5
31 1937 4.0
13 1976 4.0
4 940 3.0
125 745 1.5
120 147 4.0
247 140 3.0
40 1496 2.5
2 310 3.0
392 553 3.0
354 1 5.0
0 926 3.5
42 246 4.0
58 166 4.5
126 207 4.0
86 65 4.0
16 1230 3.0
106 4 3.5
3 798 3.5
202 164 4.0
10 471 4.0
130 142 5.0
44 652 4.0
68 214 2.0
514 468 4.0
34 386 1.0
51 323 3.0
78 

44 855 4.0
209 861 3.0
396 1363 2.0
24 100 3.5
381 14 3.5
33 1076 3.0
587 443 3.0
122 60 4.0
267 252 3.0
15 1781 3.0
281 1275 5.0
285 7 5.0
131 206 5.0
81 170 2.5
111 75 5.0
211 534 4.0
6 727 2.5
29 836 3.5
571 913 5.0
261 282 3.5
169 186 3.5
59 135 5.0
32 686 3.0
8 955 4.0
9 979 4.0
54 62 4.0
67 550 3.0
17 211 3.5
130 925 4.0
36 325 3.0
37 359 4.5
104 239 3.5
7 964 4.0
46 1332 3.0
26 980 5.0
266 1345 4.0
406 454 3.0
18 451 3.0
23 173 3.0
120 58 5.0
10 892 4.0
3 1579 2.0
259 79 4.0
227 1810 2.0
85 333 3.0
244 11 4.0
416 1434 3.0
71 232 4.0
35 372 3.0
118 1224 5.0
443 260 4.0
208 1815 4.0
35 702 5.0
89 151 4.0
50 1467 3.5
63 803 5.0
32 481 4.0
61 402 0.5
40 149 4.0
36 3 4.5
62 1457 4.0
47 985 3.5
603 152 4.0
80 420 2.0
46 669 3.0
45 477 4.0
549 641 4.0
42 1026 4.0
174 93 3.0
16 691 3.0
123 1630 5.0
395 1193 5.0
288 261 3.0
286 131 3.0
36 31 3.5
12 1157 4.5
297 468 3.0
43 629 3.0
535 6 4.0
2 1124 2.0
87 110 4.5
39 90 4.0
33 1396 5.0
4 408 4.0
177 12 4.0
1 393 3.0
14 534 4.0
1 1966 2.0
32

...

56 201 2.0
338 121 4.0
6 185 2.5
174 54 4.0
382 254 2.0
18 501 4.5
6 396 4.5
44 301 2.0
100 368 3.0
45 737 2.5
88 271 3.5
20 1665 2.0
133 566 3.5
184 277 3.0
63 13 4.0
0 1607 2.0
141 610 4.0
90 1551 3.0
73 554 5.0
92 98 4.0
45 267 4.0
490 313 4.0
315 105 4.0
57 198 3.5
156 691 4.0
384 1013 4.0
74 975 2.0
75 1207 3.5
53 939 5.0
16 709 3.0
41 403 2.0
214 6 2.0
212 619 3.5
37 538 4.5
3 1793 4.0
128 295 5.0
197 4 5.0
82 810 4.0
261 153 5.0
48 370 4.0
8 1245 4.0
111 406 5.0
0 422 3.5
23 1985 3.0
202 312 5.0
298 255 4.0
54 545 5.0
79 2 2.0
387 115 4.0
0 1154 4.0
3 1848 2.5
0 467 5.0
78 410 5.0
111 639 5.0
3 1533 4.0
379 549 3.5
188 0 5.0
72 1563 3.5
192 1271 4.5
48 1031 3.0
486 45 4.5
22 14 4.0
249 887 3.5
303 1425 4.5
3 1643 2.0
33 62 4.0
45 167 3.0
94 636 4.5
262 65 5.0
79 1510 3.0
25 18 5.0
5 128 4.5
22 155 4.0
11 1177 3.5
18 1103 3.0
137 130 4.0
113 1346 4.5
87 1452 4.5
13 137 4.0
108 270 4.5
218 157 3.0
31 58 4.0
31 1390 3.0
5 1127 3.0
289 769 3.0
85 5 4.0
2 627 4.0
276 334 3.0
105 363 

139 1204 3.5
133 664 4.5
55 995 1.0
12 1389 4.0
223 666 1.0
223 299 3.0
41 344 4.0
10 490 3.0
281 331 5.0
143 590 4.0
219 1510 5.0
283 571 4.5
43 837 4.0
415 40 3.0
74 25 3.0
8 1090 1.5
277 23 3.0
125 1333 4.0
252 6 3.0
54 22 4.0
90 1997 4.0
29 159 3.5
143 17 4.0
382 883 2.0
6 1042 3.0
25 236 3.0
319 157 4.0
470 1346 4.0
26 503 5.0
31 1488 3.0
27 1307 0.5
99 35 3.5
56 1074 2.0
68 123 4.5
22 732 4.0
94 60 4.5
78 255 4.5
11 1159 3.0
169 284 2.0
3 559 2.0
487 1 4.0
28 1226 3.0
64 1087 4.0
55 420 4.5
21 385 4.0
43 818 3.0
33 1525 3.0
225 258 5.0
4 627 4.0
45 684 3.0
15 195 3.5
429 26 4.5
30 1325 3.0
566 191 2.0
30 1259 3.0
558 537 5.0
309 421 5.0
23 626 3.5
248 213 4.0
231 33 5.0
4 319 3.5
468 386 4.0
27 858 3.0
25 1774 2.0
221 211 4.0
215 777 4.0
99 311 3.0
21 164 4.0
31 127 5.0
19 149 4.5
591 26 5.0
166 1263 3.5
99 300 4.0
14 1681 3.0
230 813 3.0
361 157 3.0
59 1050 3.0
248 728 1.5
43 92 3.0
1 652 4.0
90 253 4.0
314 18 2.5
111 63 5.0
27 259 3.5
7 22 4.0
130 329 5.0
80 29 4.0
1 149 3.5
17

207 483 3.5
158 1861 4.0
103 50 3.0
63 89 4.0
35 345 2.0
15 254 1.5
167 20 5.0
271 34 3.0
255 1695 4.5
221 1062 3.5
67 730 4.0
1 944 3.5
45 1091 3.0
7 1075 5.0
526 66 1.0
47 700 3.5
35 690 4.0
9 500 3.5
316 1886 3.0
75 10 5.0
60 544 4.0
32 191 3.0
275 144 3.5
221 528 4.0
6 1781 3.5
41 1331 1.5
178 251 3.5
7 819 5.0
58 376 3.5
46 320 3.0
40 319 5.0
147 465 3.5
291 444 4.0
273 37 4.0
394 378 5.0
30 320 3.0
50 15 4.0
18 545 3.5
299 48 4.0
244 198 3.5
551 113 4.0
40 379 4.0
208 54 4.0
52 713 4.0
289 742 5.0
73 201 3.5
73 359 4.0
313 515 2.0
55 943 3.0
99 170 3.5
262 1269 4.0
2 700 3.0
35 293 3.0
72 1547 2.5
13 19 3.0
67 796 3.5
43 134 3.0
538 463 0.5
52 1518 3.0
486 635 2.0
307 79 4.0
3 1940 3.5
135 1362 1.5
127 595 5.0
62 216 5.0
70 249 4.0
264 66 3.0
0 1720 2.5
92 1197 5.0
30 1437 4.0
37 820 3.5
8 235 4.0
251 971 5.0
245 375 4.0
325 6 4.0
12 183 4.0
14 1085 4.0
74 255 2.0
34 169 4.5
0 1906 3.5
146 1033 4.0
36 787 3.0
32 209 4.5
64 806 2.0
113 1411 4.0
42 203 4.0
24 785 5.0
122 336 4.0
47

259 49 2.0
31 612 3.0
162 599 3.0
594 41 5.0
338 208 5.0
16 1645 4.0
175 120 3.0
6 414 0.5
62 364 5.0
3 1393 4.0
129 193 4.5
26 1393 4.0
397 49 3.0
17 40 3.5
519 0 5.0
131 1389 5.0
107 475 4.0
280 38 5.0
378 1159 3.0
246 1059 4.0
256 497 4.0
178 145 2.5
41 184 4.0
95 295 5.0
113 1157 4.0
137 136 4.0
95 136 3.0
156 135 4.0
87 1943 4.5
0 572 5.0
7 955 2.5
3 289 4.0
16 86 1.0
103 404 5.0
57 36 4.0
258 46 5.0
33 1538 1.0
194 285 3.5
38 381 2.0
87 68 5.0
32 1811 4.0
309 17 4.0
302 1335 3.0
69 92 3.5
327 77 3.5
38 232 2.5
40 48 3.5
170 41 4.0
217 692 1.0
37 889 3.0
446 91 5.0
6 27 3.5
158 1961 3.0
144 921 5.0
303 243 4.0
132 50 5.0
271 112 4.0
431 465 3.5
46 638 3.0
171 410 4.0
284 254 4.0
59 1457 4.0
20 283 3.5
131 655 5.0
24 811 3.5
6 1771 3.5
108 210 3.5
306 752 4.0
195 6 4.0
294 29 5.0
109 21 5.0
30 809 2.0
584 1341 2.0
2 1355 2.0
25 431 3.0
117 77 4.5
32 264 4.0
221 36 3.0
48 24 4.0
25 1522 1.0
64 1344 1.0
87 739 4.0
7 1975 4.0
59 292 4.0
164 347 3.0
35 43 3.5
51 299 3.0
21 1884 3.5
1 2

62 115 4.0
52 93 2.5
24 767 4.0
5 243 5.0
210 1456 4.0
133 680 4.5
25 1779 2.0
426 152 4.0
21 463 2.5
121 1056 4.0
2 1230 3.5
191 155 4.0
70 1534 3.5
452 4 5.0
133 66 4.5
426 502 5.0
27 276 3.5
114 57 2.0
74 219 3.0
9 8 4.0
12 1492 3.0
95 773 4.0
121 232 1.0
39 1043 4.0
114 231 4.0
58 351 3.5
6 1540 3.0
8 147 4.0
239 140 3.0
189 36 3.0
5 1466 4.0
149 180 0.5
11 295 4.5
12 1083 4.5
26 1557 4.0
562 8 5.0
34 112 2.0
86 1513 3.0
220 33 2.5
28 0 5.0
211 21 3.0
248 12 3.0
4 1603 3.0
162 166 3.0
6 799 4.5
47 760 2.5
14 1166 3.0
29 34 4.0
88 629 3.0
70 1519 4.0
240 135 5.0
458 75 3.0
73 1257 5.0
92 44 3.5
17 113 4.0
114 124 5.0
67 456 4.0
119 214 2.0
119 1251 4.0
17 662 5.0
220 806 0.5
91 1014 3.5
197 569 2.0
80 319 5.0
1 1938 3.0
64 750 2.0
139 767 3.5
196 631 3.5
35 241 3.0
79 1429 4.0
133 7 4.0
104 58 2.5
112 24 4.0
105 209 3.5
78 492 4.5
266 1852 5.0
4 69 4.0
0 1791 4.0
45 1705 5.0
37 51 4.5
2 828 4.0
103 312 4.0
61 66 0.5
43 676 3.5
122 84 4.0
34 83 5.0
80 1528 5.0
164 377 5.0
326 1152 3.

### Item to Item Collaborative

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

In [5]:
# load in the data
if not os.path.exists(path + 'user2movie.json') or \
    not os.path.exists(path + 'movie2user.json') or \
    not os.path.exists(path + 'usermovie2rating.json') or \
    not os.path.exists(path + 'usermovie2rating_test.json'):
    import preprocess2dict

In [6]:
with open(path + 'user2movie.json', 'rb') as f:
    user2movie = pickle.load(f)

with open(path + 'movie2user.json', 'rb') as f:
    movie2user = pickle.load(f)

with open(path + 'usermovie2rating.json', 'rb') as f:
    usermovie2rating = pickle.load(f)
    
with open(path + 'usermovie2rating_test.json', 'rb') as f:
    usermovie2rating_test = pickle.load(f)

In [8]:
N = np.max(list(user2movie.keys())) + 1
# the test set may contain movies the train set doesn't have data on
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

#if M > 2000:
#    print("N =", N, "are you sure you want to continue?")
#    print("Comment out these lines if so...")
#    exit()

N: 610 M: 2000


In [9]:
# to find the user similarities, you have to do O(M^2 * N) calculations!
# in the "real-world" you'd want to parallelize this
# note: we really only have to do half the calculations, since w_ij is symmetric
K = 20 # number of neighbors we'd like to consider
limit = 5 # number of common movies users must have in common in order to consider
neighbors = [] # store neighbors in this list
averages = [] # each item's average rating for later use
deviations = [] # each item's deviation for later use

In [10]:
for i in range(M):
# find the K closest items to item i
    users_i = movie2user[i]
    users_i_set = set(users_i)

    # calculate avg and deviation
    ratings_i = { user:usermovie2rating[(user, i)] for user in users_i }
    avg_i = np.mean(list(ratings_i.values()))
    dev_i = { user:(rating - avg_i) for user, rating in ratings_i.items() }
    dev_i_values = np.array(list(dev_i.values()))
    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))

    # save these for later use
    averages.append(avg_i)
    deviations.append(dev_i)

    sl = SortedList()
    for j in range(M):
        # don't include yourself
        if j != i:
            users_j = movie2user[j]
            users_j_set = set(users_j)
            common_users = (users_i_set & users_j_set) # intersection
            if len(common_users) > limit:
                # calculate avg and deviation
                ratings_j = { user:usermovie2rating[(user, j)] for user in users_j }
                avg_j = np.mean(list(ratings_j.values()))
                dev_j = { user:(rating - avg_j) for user, rating in ratings_j.items() }
                dev_j_values = np.array(list(dev_j.values()))
                sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))

                # calculate correlation coefficient
                numerator = sum(dev_i[m]*dev_j[m] for m in common_users)
                w_ij = numerator / (sigma_i * sigma_j)

                # insert into sorted list and truncate
                # negate weight, because list is sorted ascending
                # maximum value (1) is "closest"
                sl.add((-w_ij, j))
                if len(sl) > K:
                    del sl[-1]

    # store the neighbors
    neighbors.append(sl)

    # print out useful things
    if i % 1 == 0:
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999


In [11]:
# using neighbors, calculate train and test MSE

def predict(i, u):
    # calculate the weighted sum of deviations
    numerator = 0
    denominator = 0
    for neg_w, j in neighbors[i]:
        # remember, the weight is stored as its negative
        # so the negative of the negative weight is the positive weight
        try:
            numerator += -neg_w * deviations[j][u]
            denominator += abs(neg_w)
        except KeyError:
            # neighbor may not have been rated by the same user
            # don't want to do dictionary lookup twice
            # so just throw exception
            pass

    if denominator == 0:
        prediction = averages[i]
    else:
        prediction = numerator / denominator + averages[i]
    prediction = min(5, prediction)
    prediction = max(0.5, prediction) # min rating is 0.5
    return prediction

In [12]:
train_predictions = []
train_targets = []
for (u, m), target in usermovie2rating.items():
    # calculate the prediction for this movie
    prediction = predict(m, u)

    # save the prediction and target
    train_predictions.append(prediction)
    train_targets.append(target)

In [13]:
test_predictions = []
test_targets = []
# same thing for test set
for (u, m), target in usermovie2rating_test.items():
    # calculate the prediction for this movie
    prediction = predict(m, u)

    # save the prediction and target
    test_predictions.append(prediction)
    test_targets.append(target)

In [14]:
# calculate accuracy
def mse(p, t):
    p = np.array(p)
    t = np.array(t)
    return np.mean((p - t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

train mse: 0.39758266566991735
test mse: 0.8058935834936014
